# 전처리(2) - 품사 태깅 및 필터링
- 최종 POS Tagger 및  : bareun 형태소 분석 API
- 품사 필터링 : AntConc에서 키워드 리스트를 생성하기 위해, 품사를 필터링한 후 각 단어 옆에 품사 태그를 붙이는 방식으로 데이터를 변환하였음

- 남긴 품사 : 명사(NNG, NNP), 동사(VV), 형용사(VA) (조정 가능함)


In [ ]:
import tensorflow as tf
print("TensorFlow name : ", tf.__name__)

TensorFlow name :  tensorflow


In [ ]:
import os
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
# import wandb
# from wandb.keras import WandbCallback
from sklearn.preprocessing import LabelEncoder
import re
import numpy as np
import csv

print("다음 단계로")

다음 단계로


### 바른 API 설치

In [ ]:
# 바른의 리눅스 설치본 다운로드

!curl -LJks -H "uname:$(uname -a)" https://bareun.ai/api/get -o bareun-linux.deb
!ls

bareun-linux.deb  drive  sample_data


In [ ]:
# 바른 설치

!uname -a
!dpkg -i bareun-linux.deb

Linux c65693c3c60a 6.1.85+ #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux
Selecting previously unselected package bareun.
(Reading database ... 123620 files and directories currently installed.)
Preparing to unpack bareun-linux.deb ...
Unpacking bareun (2.3.0) ...
Setting up bareun (2.3.0) ...
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
Configuration file /etc/systemd/system/bareun.service is marked executable. Please remove executable permission bits. Proceeding anyway.
Created symlink /etc/systemd/system/multi-user.target.wants/bareun.service → /etc/systemd/system/bareun.service.


In [ ]:
# 바른 모델이 GPU를 사용하기 위해서 다운받는 tensorflow 라이브러리

!curl -O https://storage.googleapis.com/tensorflow/libtensorflow/libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz
!tar -C /opt/bareun -xzf libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  392M  100  392M    0     0  19.8M      0  0:00:19  0:00:19 --:--:-- 22.5M


In [ ]:
%env BAREUN_ROOT="/opt/bareun"
%env LD_LIBRARY_PATH="/opt/bareun/lib"
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" nohup /opt/bareun/bin/bareun&

env: BAREUN_ROOT="/opt/bareun"
env: LD_LIBRARY_PATH="/opt/bareun/lib"
nohup: appending output to 'nohup.out'


In [ ]:
!ps -ef | grep bareun

root        2086       1  1 14:22 ?        00:00:18 /opt/bareun/bin/bareun
root        9496       1  0 14:53 ?        00:00:00 /opt/bareun/bin/bareun
root        9501    1006  0 14:53 ?        00:00:00 /bin/bash -c ps -ef | grep bareun
root        9503    9501  0 14:53 ?        00:00:00 grep bareun


In [ ]:
# Yeonwoo's API KEY

!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" /opt/bareun/bin/bareun -reg koba-MDIHR4Q-BQUUMOY-TOGE42A-NDESPKY

2024-10-07 14:53:11.733368: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bareun/lib
Use config file: /opt/bareun/config/bareun.json

------
200 OK
sucessfully registered koba-MDIHR4Q-BQUUMOY-TOGE42A-NDESPKY


In [ ]:
!pip install -U bareunpy

In [ ]:
import sys
import bareunpy as brn
import google.protobuf.text_format as tf

from bareunpy import Tagger   # 이하 추가
from bareunpy import Tokenizer

In [ ]:
# 아래에 "https://bareun.ai/"에서 이메일 인증 후 발급받은 API KEY("koba-...")를 입력해주세요. "로그인-내정보 확인"

API_KEY = "koba-MDIHR4Q-BQUUMOY-TOGE42A-NDESPKY" # <- 본인의 API KEY로 교체

# sample code : 정상 작동 확인
t = brn.Tagger(API_KEY, "localhost", 5656)
res = t.tags(["안녕하세요. 정말 좋은 날씨네요."])
m = res.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)

sentences {
  text {
    content: "안녕하세요. 정말 좋은 날씨네요."
    length: 18
  }
  tokens {
    text {
      content: "안녕하세요."
      length: 6
    }
    morphemes {
      text {
        content: "안녕하"
        length: 3
      }
      tag: VA
      out_of_vocab: IN_BUILTIN_DICT
    }
    morphemes {
      text {
        content: "시"
        begin_offset: 3
        length: 1
      }
      tag: EP
      probability: 0.9998893
    }
    morphemes {
      text {
        content: "어요"
        begin_offset: 3
        length: 2
      }
      tag: EF
      probability: 0.9899567
    }
    morphemes {
      text {
        content: "."
        begin_offset: 5
        length: 1
      }
      tag: SF
      probability: 0.99979943
    }
    lemma: "안녕하"
    tagged: "안녕하/VA+시/EP+어요/EF+./SF"
  }
  tokens {
    text {
      content: "정말"
      begin_offset: 7
      length: 2
    }
    morphemes {
      text {
        content: "정말"
        begin_offset: 7
        length: 2
      }
      tag: MAG
      probabili

## 데이터 불러오기

In [ ]:
# 맞춤법 검사 및 교정된 train data 불러오기

train_data_path = '/content/drive/MyDrive/aiffel/dktc/data/filtered_normal/clined_filtered_spell_checked_normal_conv.csv'
train_data = pd.read_csv(train_data_path)
train_data.head()

ParserError: Error tokenizing data. C error: Expected 2 fields in line 11, saw 4


In [ ]:
train_data.info()

In [ ]:
train_data.isnull().sum()

In [ ]:
df_train_before = train_data.copy()

In [ ]:
# 누락 데이터 count 위한 함수 정의

def convert_empty_to_nan(df):
    # DataFrame 내의 빈 문자열을 NaN으로 변환
    df.replace('', np.nan, inplace=True)
    return df

def count_empty_rows(df):
    # 각 행의 모든 값이 NaN인 경우를 찾음
    empty_rows = df.isnull().all(axis=1)

    # 빈 행의 개수 출력
    num_empty_rows = empty_rows.sum()
    return num_empty_rows

In [ ]:
df_train = convert_empty_to_nan(df_train_before)
df_train = df_train.dropna()
n = count_empty_rows(df_train)
n

In [ ]:
normal_corpus = df_train.copy()

## train data : 클래스(레이블) 인코딩

In [ ]:
# train data에 적용

CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화']

encoder = LabelEncoder()
encoder.fit(CLASS_NAMES)

train_data['class'] = encoder.transform(train_data['class'])

corpus = train_data["conversation"]

KeyError: 'class'

In [ ]:
corpus.head()

NameError: name 'corpus' is not defined

## POS Tagger

In [ ]:
# 태깅 코드
tagger = brn.Tagger(API_KEY, "localhost", 5656)  # 위의 변수명('t')과 일치시킬 경우, 생략하여도 무방함

### sample code : 사용자 지정 단어 사전 (생략)

In [ ]:
# custom dictionary
cust_dic = tagger.custom_dict("my")
cust_dic.copy_np_set({'내고유명사', '우리집고유명사'})
cust_dic.copy_cp_set({'코로나19'})
cust_dic.copy_cp_caret_set({'코로나^백신', '독감^백신'})
cust_dic.update()

# laod prev custom dict
cust_dict2 = tagger.custom_dict("my")
cust_dict2.load()

tagger.set_domain('my')
tagger.pos('코로나19는 언제 끝날까요?')

[('코로나19', 'NNG'),
 ('는', 'JX'),
 ('언제', 'MAG'),
 ('끝나', 'VV'),
 ('ㄹ까', 'EF'),
 ('요', 'JX'),
 ('?', 'SF')]

### 함수화 - json 객체 호출, 품사 getter, corpus 전체 태깅

In [ ]:
def tagging(tagger, sentences):
    res = tagger.tags([sentences])  # 문장 전체 태깅
    return res  # res 객체 반환


def get_json(res): # 객체를 json 형식으로 변환
    jo = res.as_json()
    return jo


def get_pos(res):  # 품사 태깅된 res instance로부터 형태소 추출
    jo = get_json(res)
    pa = res.pos()
    ma = res.morphs()
    na = res.nouns()
    va = res.verbs()
    return pa, ma, na, va, jo


def corpus_tagging(corpus, tagger):  # corpus 전체, 데이터(row) 내부의 line 단위로 품사 태깅 객체 생성
    tagged_data = []  # 태깅된 데이터를 저장할 리스트

    for i in range(len(corpus)):
        sentences = corpus.iloc[i].split('\n')  # '\n' 기준으로 문장 분리  # 수정함 : corpus[i] -> corpus.iloc[i]
        row_results = []  # 각 행에 대한 결과를 저장할 리스트

        for sentence in sentences:
            if sentence.strip():  # 빈 문장 제외
                res = tagging(tagger, sentence)
                pa, ma, na, va, jo = get_pos(res)  # 품사 태깅 결과
                row_results.append({
                    'sentence': sentence,
                    'pos': pa,
                    # 'morphs': ma,  # 필요시 추가하여 사용
                    # 'nouns': na,
                    # 'verbs': va,
                    # 'json': jo
                })  # 태깅 결과를 딕셔너리로 저장

        tagged_data.append(row_results)  # 각 행의 결과를 추가

    return tagged_data  # 태깅된 데이터 반환


def instance_info(res, i): # 필요시 사용
    m = res.msg()  # res에서 msg() 호출
    print(f'length of tokens in sentences[{i}] is {len(m.sentences)}')
    if m.sentences:
        print(f'length of morphemes of first token in sentences[{i}] is {len(m.sentences[0].tokens[0].morphemes)}')
        print(f'lemma of first token in sentences[{i}] is {m.sentences[0].tokens[0].lemma}')
        print(f'first morph of first token in sentences[{i}] is {m.sentences[0].tokens[0].morphemes[0].text.content}')
        print(f'tag of first morph of first token in sentences[{i}] is {m.sentences[0].tokens[0].morphemes[0].tag}')

## Filtering POS
- 유의미한 품사만 남긴(태깅된) text 데이터 추출

In [ ]:
def filter_pos(tagged_data, pos_to_keep):  # 품사 필터링 후 각 단어 옆에 품사 태그 붙여 반환함
    filtered_data = []

    for block in tagged_data:  # 여러 문장 블록을 포함하는 리스트
        for entry in block:  # 각 문장별로 필터링
            filtered_tokens = [
                f"{morph}/{pos}" for morph, pos in entry['pos'] if pos in pos_to_keep
            ]
            filtered_data.append(' '.join(filtered_tokens))

    return filtered_data


def save_to_csv(filtered_data, filename):  # 데이터 저장
    # Colab에서는 상대 경로를 사용해 저장
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Filtered Morphs"])  # 헤더 생성

        for filtered_sentence in filtered_data:
            writer.writerow([filtered_sentence])


In [ ]:
def apply_tagging_and_filter(conversations_class, tagger, pos_to_keep):

    filtered_conversations = []  # 필터링된 데이터를 저장할 리스트

    # 1. 품사 태깅 수행
    tagged_data = corpus_tagging(conversations_class.tolist(), tagger)  # Series를 리스트로 변환

    # 2. filter_pos 함수 적용
    filtered_data = filter_pos(tagged_data, pos_to_keep)

    # 3. 결과 저장
    filtered_conversations.append(filtered_data)

    return filtered_conversations



In [ ]:
# 필터링할 품사 선택 (예시: 명사(NNG일반, NNP고유), 동사(VV), 형용사(VA))
pos_to_keep = ['NNG', 'NNP', 'VV', 'VA']

In [ ]:
type(normal_corpus)

pandas.core.frame.DataFrame

In [ ]:
# train_data class
# .txt 형식으로 저장된 list 파일을 코랩에서 읽어오기
filtered_d_path_0 = '/content/drive/MyDrive/aiffel/dktc/data/filtered_train/conv_text_class_0.txt'
filtered_d_path_1 = '/content/drive/MyDrive/aiffel/dktc/data/filtered_train/conv_text_class_1.txt'


with open(filtered_d_path_0, 'r', encoding='utf-8') as f:
    f_data_c0 = f.readlines()

# 파일 내용 출력
print(f_data_c0)

['[\'어 이   거 기  예 ? ?  너   말 이 야   너 .\', \'이 리   오 라 고  무 슨   일 .\', \'너   옷   좋 아 보 인 다 ?  얘   돈   좀   있 나 봐  아 네 에 요 .\', \'돈   없 어 요  뒤 져 서   나 오 면   넌   죽 는 다  오 늘   피 시 방   콜 ?  콜 .\', \'마 지 막   기 회 다 .\', \'있 는 거   다   내 놔  정 말   없 어 요 저 기 요   혹 시   날 이   너 무   뜨 겁 잖 아 요 ?   저 희   회 사 에 서   이   선 크 림   파 는 데   한   번   손 등 에   발 라 보 실 래 요 ?  아   진 짜 요 ?   안   그 래 도   선 크 림   필 요 해 서   알 아 보 던   중 인 데   한   번   발 라   볼 게 요 !  여 기   한   번   발 라 보 세 요 .\', \'진 짜   성 분 도   좋 고   다   좋 아 요 .\', \'음 .\', \'성 분 이   좋 다 고   하 셔 서   좋 은 거   같 기 는   한 데   제   피 부 에   맞 지   안 나 봐 요 .\', \'피 부 가   따 끔 거 리 네 요 .\', \'이 번 에   진 짜   열 심 히   연 구 해 서   만 든 건 데   피 부 가   많 이   예 민 하 신 가 봐 요 .\', \'네   많 이   예 민 해 요 .\', \'그 럼   많 이   파 시 고   안 녕 히   계 세 요 .\', \'아 네   저 기 요   돈   안 내 요 ?  네 ?   발 라 보 는 것 도   돈   내 야   하 나 요 ?  그 럼   이 거   누 구 한 테   팔 아 요 ?   당 신 이   바 른 거 를 ?  아 네   먼 저   발 라   보 시 라 고   하 셨 잖 아 요 .\', \'먼 저   권 유 해 놓 고   사 라 고   강 매 하 는 거   갈 취 인 거   몰 라 요 ?  내 가   안   사 도   된 다 

In [ ]:
with open(filtered_d_path_1, 'r', encoding='utf-8') as f:
    f_data_c1 = f.readlines()

# 파일 내용 출력
print(f_data_c1)

['[\'너   되 게   귀 여 운 거   알 지 ?   나 보 다   작 은   남 자 는   첨 봤 어 .\', \'그 만 해 .\', \'네 들   놀 리 는 거   재 미 없 어 .\', \'지 영 아   너 가   키   1 6 0 이 지 ?   그 럼   재 는   1 6 0 도   안 돼 는 거 네 ?  너   군 대 도   안 가 고   좋 겠 다 .\', \'네 들 이   나   작 은 데   보 태 준 거   있 냐 ?  난 쟁 이 들 도   장 가 가 고 하 던 데 .\', \'너 도   희 망 을   가 져 봐    더 이 상   하 지 마 라 .\', \'그   키 크 는   수 술 도   있 대 잖 아 ?   네 네   엄 마 는   그 거   안 해 주 디 ?  나 람   해 줬 어 .\', \'저   키 로   어 찌 살 아 .\', \'제 발   그 만   괴 롭 히 라 고 ! 3 5 번   손 님   아 이 스 커 피   두 잔 나 왔 습 네 다  아 이 스 커 피 ?    네   맛 있 게 드 세 요  저 기 요   아 가 씨   나 는   아 아 스   시 킨 적 이   없 는 데 ?  아   분 명   오 늘   날 이   더 우 시 다 고   아 이 스 로   시 키 셨 는 데 요  내 가   그 랫 어 ?    네 분 명 히 .\', \'아 그 런   기 억 이   없 는 데 ?   아 가 씨 가 잘 못 안 거   아 네 야 ?  아 네 요 .\', \'오 늘   손 님 이   첫   주 문 이 라   확 실 히   기 억 하 고   있 습 네 다  아 가 씨 .\', \'왜 이 이렇 게   유 도 리 가   없 이   굴 어   그 냥   아   제 가   잘 못   주 문   받 았 습 네 다   하 면   되 지 ?  .\', \'네 ? .\', \'어 휴   유 도 리   없 어   그 냥   마 실 게 경 비 아 저 씨   내 차 에   경 고 장   붙 였 어 요 ?  내 가   여 기   몇 년 째   사

In [ ]:
# 정규표현식 적용 함수

def preprocess_sentence(sentence):     # 추후 사용자 지정 단어 사전 구축하기 : 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ', '^^', '넹넹', '웅웅' 등
    sentence = sentence.strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ'의 반복을 각각 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ'로 통일
    sentence = re.sub(r"(ㅠ)+", "ㅠㅠ", sentence)  # 'ㅠㅠ' 이상 반복을 'ㅠㅠ'로
    sentence = re.sub(r"(ㅎ)+", "ㅎㅎ", sentence)  # 'ㅎㅎ' 이상 반복을 'ㅎㅎ'로
    sentence = re.sub(r"(ㅋ)+", "ㅋㅋ", sentence)  # 'ㅋㅋ' 이상 반복을 'ㅋㅋ'로
    # 한글, 알파벳, 문장 부호 및 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ', '^^'는 남기고 나머지는 공백으로 대체
    sentence = re.sub(r"[^가-힣a-zA-Z?.!,ㅠㅠㅎㅎㅋㅋ^^]+", " ", sentence)

    sentence = sentence.strip()
    return sentence


def load_conversations_from_df(corpus_df):
    conv_data = []

    for index, row in corpus_df.iterrows():  # 수정된 부분
        conversations = row['conversation']  # 수정된 부분
        conv_data.append(preprocess_sentence(conversations))  # 전처리 함수 적용
    return conv_data

In [ ]:
f_data_c0 = pd.DataFrame(f_data_c0, columns=['conversation'])
f_data_c1 = pd.DataFrame(f_data_c1, columns=['conversation'])

conv_f_data_c0 = load_conversations_from_df(f_data_c0)
conv_f_data_c1 = load_conversations_from_df(f_data_c1)


In [ ]:
# 데이터가 'tagged_data'라는 변수에 있다고 가정
# sample code (코퍼스 태깅)

# 함수 재정의
def corpus_tagging(corpus, tagger):  # corpus 전체, 데이터(row) 내부의 line 단위로 품사 태깅 객체 생성
    tagged_data = []  # 태깅된 데이터를 저장할 리스트

    for i in range(len(corpus)):
        sentences = corpus[i].split('\n')  # '\n' 기준으로 문장 분리
        row_results = []  # 각 행에 대한 결과를 저장할 리스트

        for sentence in sentences:
            if sentence.strip():  # 빈 문장 제외
                res = tagging(tagger, sentence)
                pa, ma, na, va, jo = get_pos(res)  # 품사 태깅 결과
                row_results.append({
                    'sentence': sentence,
                    'pos': pa,
                    # 'morphs': ma,  # 필요시 추가하여 사용
                    # 'nouns': na,
                    # 'verbs': va,
                    # 'json': jo
                })  # 태깅 결과를 딕셔너리로 저장

        tagged_data.append(row_results)  # 각 행의 결과를 추가

    return tagged_data  # 태깅된 데이터 반환

# corpus00 = normal_corpus.iloc[:, 0]  # 일반대화 데이터셋
tagged_results00 = corpus_tagging(conv_f_data_c0, tagger)

tagged_data00 = tagged_results00

# 필요한 품사만 필터링
filtered_results00 = filter_pos(tagged_data00, pos_to_keep)

In [ ]:
tagged_results01 = corpus_tagging(conv_f_data_c1, tagger)
tagged_data01 = tagged_results01
# 필요한 품사만 필터링
filtered_results01 = filter_pos(tagged_data01, pos_to_keep)

In [ ]:
conv_f_data_c2 =
conv_f_data_c3 =

tagged_results02 = corpus_tagging(conv_f_data_c2, tagger)
tagged_results03 = corpus_tagging(conv_f_data_c3, tagger)

tagged_data02 = tagged_results02
tagged_data03 = tagged_results03

# 필요한 품사만 필터링
filtered_results02 = filter_pos(tagged_data03, pos_to_keep)
filtered_results03 = filter_pos(tagged_data03, pos_to_keep)

In [ ]:
# 필터링 결과를 CSV로 저장 (Filtered Morphs만 저장)
save_to_csv(filtered_results00, '/content/drive/MyDrive/aiffel/dktc/result/filtered_pos_class_0.csv')
save_to_csv(filtered_results01, '/content/drive/MyDrive/aiffel/dktc/result/filtered_pos_class_1.csv')

In [ ]:
save_to_csv(filtered_results02, '/content/drive/MyDrive/aiffel/dktc/result/filtered_pos_class_2.csv')
save_to_csv(filtered_results03, '/content/drive/MyDrive/aiffel/dktc/result/filtered_pos_class_3.csv')

# 전처리(3) - 형태소 분석, Tokenizer
- 최종 형태소 분석기 : bareun API

In [ ]:
# 토크나이징 코드

# API_KEY = "koba-MDIHR4Q-BQUUMOY-TOGE42A-NDESPKY" # Yeonwoo's API KEY

tokenizer = brn.Tokenizer(API_KEY, "localhost", 5656)   # tokenizer 생성


In [ ]:
def tagging_and_tokenizing(tagger, tokenizer, sentences):
    # 품사 태깅
    tagged_res = tagger.tags([sentences])
    pa, ma, na, va, jo = get_pos(tagged_res)

    # 토큰화
    tokenized = tokenizer.tokenize_list([sentences])
    tokenized_msg = tokenized.msg()

    # tokenized 결과 추출
    tokens_info = []
    for token in tokenized_msg.sentences[0].tokens:
        tokens_info.append({
            'text': token.text.content,
            'segments': [seg.text.content for seg in token.segments],
            'tagged': token.tagged
        })

    return {
        'pos': pa,
        'morphs': ma,
        'nouns': na,
        'verbs': va,
        'json': jo,
        'tokens_info': tokens_info
    }

def corpus_tagging_and_tokenizing(corpus, tagger, tokenizer):
    tagged_data = []  # 태깅된 데이터를 저장할 리스트

    for i in range(len(corpus)):
        sentences = corpus.iloc[i, 0].split('\n')  # '\n' 기준으로 문장 분리
        row_results = []  # 각 행에 대한 결과를 저장할 리스트
        for sentence in sentences:
            if sentence.strip():  # 빈 문장 제외
                result = tagging_and_tokenizing(tagger, tokenizer, sentence)
                row_results.append({
                    'sentence': sentence,
                    'pos': result['pos'],
                    'morphs': result['morphs'],
                    'nouns': result['nouns'],
                    'verbs': result['verbs'],
                    'json': result['json'],
                    'tokens_info': result['tokens_info']
                })

        tagged_data.append(row_results)  # 각 행의 결과를 추가

    return tagged_data  # 태깅된 데이터 반환



In [ ]:
normal_corpus

,0
2,넌 계속 부산에서 살 거냐 나는 부산 안 떠나고 싶어 ㅠㅠ 부산은 너 싫다는데? 하...
3,나 며칠 전에 생치 보고 왔음 오 그거 마블 새 영화? 응 맞아 첫 번째 아시안 히...
6,이제 백신 접종 차까지 마친 사람들은 시까지 식당이나 술집을 이용할 수 있다네 그래...
8,오늘 길 가는데 너 선임 만났음 키키 엥? 어디서 걔를 만나? 키키 영화 보고 나오...
9,제주도 여행 가고 싶어요 이번에 가게 되면 코스를 잘 짜고 싶다 코스도 코슨데 식당...
...,...
3990,콘서트나 뮤지컬 보는 거 좋아해? 콘서트 보러 가고 싶어 키키 본 적 있어? 응 박...
3993,날씨 예보 보니까 가을 옷 장만 안 해도 될 거 같아요 키키 아 키키 그렇지? 키키...
3994,아 머리 시술하고 싶다 모기의 다리를 내 눈썹에 붙여줘 키키 무슨 말이야 진짜 대단...
3996,우리 남편은 병장 때 월급 만 원이었대. . . 내도 그 정도였다 만 원? 요즘 얼...


In [ ]:
# 태거 및 토크나이저 객체 생성 (가정된 API 호출)

corpus_tt = normal_corpus
tagged_and_tokenized_results = corpus_tagging_and_tokenizing(corpus_tt, tagger, tokenizer)

# 결과 확인
print(tagged_and_tokenized_results)

Buffered data was truncated after reaching the output size limit.

In [ ]:
tagged_and_tokenized_results[:10]

[[{'sentence': '넌 계속 부산에서 살 거냐 나는 부산 안 떠나고 싶어 ㅠㅠ 부산은 너 싫다는데? 하하 바다 없음 못 살아. . . 빌붙어 살래 키키 오 짝사랑할 만큼 가치 있는 거가 밤바다 좋아하는 인 맞아 나도 바다 없이 못 살아 ㅠㅠ^ 보고 싶을 때 볼 수 있는 게 얼마나 좋아 근데 산도 좋아 여름에 춥고 겨울에 따뜻하고 그래서 부산 안 떠날래. . . 너는? 난 갈 수 있으면 가야지 근데 확실히 서울은 클래스가 다르더라 사람은 크게 놀아야 해 ㅎㅎ 서울 생활 무서워',
   'pos': [('너', 'NP'),
    ('ㄴ', 'JX'),
    ('계속', 'MAG'),
    ('부산', 'NNP'),
    ('에서', 'JKB'),
    ('살', 'VV'),
    ('ㄹ', 'ETM'),
    ('거', 'NNB'),
    ('이', 'VCP'),
    ('냐', 'EF'),
    ('나', 'NP'),
    ('는', 'JX'),
    ('부산', 'NNP'),
    ('안', 'MAG'),
    ('떠나', 'VV'),
    ('고', 'EC'),
    ('싶', 'VX'),
    ('어', 'EC'),
    ('ㅠㅠ', 'NNG'),
    ('부산', 'NNP'),
    ('은', 'JX'),
    ('너', 'NP'),
    ('싫', 'VA'),
    ('다는데', 'EF'),
    ('?', 'SF'),
    ('하하', 'IC'),
    ('바다', 'NNG'),
    ('없', 'VA'),
    ('음', 'ETN'),
    ('못', 'MAG'),
    ('살', 'VV'),
    ('아', 'EF'),
    ('.', 'SF'),
    ('.', 'SF'),
    ('.', 'SF'),
    ('빌붙', 'VV'),
    ('어', 'EC'),
    ('살래', 'MAG'),
    ('키키', 'MAG'),
    ('오', 'NR'),
    ('짝사랑

In [ ]:
type(conv_f_data_c0)

list

In [ ]:
## POS filtered train data -> tagging and tokenizer

corpus_train_0 = pd.DataFrame(conv_f_data_c0)
corpus_train_1 = pd.DataFrame(conv_f_data_c1)

tagged_and_tokenized_results_C0 = corpus_tagging_and_tokenizing(corpus_train_0, tagger, tokenizer)
tagged_and_tokenized_results_C1 = corpus_tagging_and_tokenizing(corpus_train_1, tagger, tokenizer)

# 결과 확인
print(tagged_and_tokenized_results_C0[:10])
print('-'*30)
print(tagged_and_tokenized_results_C1[:10])

Buffered data was truncated after reaching the output size limit.